# Compute centrality measures

In [4]:
import igraph as ig
import networkx as nx
import numpy as np
import pandas as pd
import operator
import osmnx as ox
from sqlalchemy import create_engine

%matplotlib inline
ox.config(log_console=True)
print(ox.__version__)
print(ig.__version__)

1.1.12021-10-25 12:28:14 Configured OSMnx 1.1.1

0.9.7
2021-10-25 12:28:14 HTTP response caching is on


In [5]:
def compute_node_closeness_per_city(city_name, engine):
    weight = 'length'
    # create networkx graph
    G_nx = ox.graph_from_place(city_name, network_type='walk')
    osmids = list(G_nx.nodes)
    G_nx = nx.relabel.convert_node_labels_to_integers(G_nx)

    # give each node its original osmid as attribute since we relabeled them
    osmid_values = {k:v for k, v in zip(G_nx.nodes, osmids)}
    nx.set_node_attributes(G_nx, osmid_values, 'osmid')
    # convert nodes to lines

    print("Convert to igraph ...")

    # %%time
    # convert networkx graph to igraph
    G_ig = ig.Graph(directed=True)
    G_ig.add_vertices(G_nx.nodes)
    G_ig.add_edges(G_nx.edges())
    G_ig.vs['osmid'] = osmids
    G_ig.es[weight] = list(nx.get_edge_attributes(G_nx, weight).values())

    assert len(G_nx.nodes()) == G_ig.vcount()
    assert len(G_nx.edges()) == G_ig.ecount()
    print("Calculating closeness ...")

    closeness1 = G_ig.closeness(vertices=None, mode='ALL', cutoff=None, weights=weight, normalized=True)
    # nx.set_node_attributes(G_nx, closeness1, 'node_closeness')
    print("Convert to dataframe closeness ...")
    gdf_nodes = ox.utils_graph.graph_to_gdfs(G_nx, nodes=True, edges=False, node_geometry=True, fill_edge_geometry=False)
    df_nodes = pd.DataFrame({'osmid': G_ig.vs["osmid"], 'node_closeness':closeness1})
    gdf_nodes = gdf_nodes.reset_index(drop=True)
    gdf_res = pd.merge(gdf_nodes, df_nodes, left_on='osmid', right_on='osmid', how='left')
    gdf_res.to_postgis("street_node_centr_" + city_name[0:3] , schema=city_name, con=engine)

In [6]:
def compute_edge_betweenness_per_city(city_name, engine):
    weight = 'length'
    # create networkx graph
    G_nx = ox.graph_from_place(city_name, network_type='walk')
    osmids = list(G_nx.edges)
    G_nx = nx.relabel.convert_node_labels_to_integers(G_nx)

    # G_nx = ox.project_graph(G_nx)
    osmid_values = {k:v for k, v in zip(G_nx.edges, osmids)}

    # # give each node its original osmid as attribute since we relabeled them
    nx.set_edge_attributes(G_nx, osmid_values, 'osmid')
    print("Convert to igraph ...")

    # %%time
    # convert networkx graph to igraph
    G_ig = ig.Graph(directed=True)
    G_ig.add_vertices(G_nx.nodes)
    G_ig.add_edges(G_nx.edges())
    G_ig.es['osmid'] = osmids
    G_ig.es[weight] = list(nx.get_edge_attributes(G_nx, weight).values())

    assert len(G_nx.nodes()) == G_ig.vcount()
    assert len(G_nx.edges()) == G_ig.ecount()
    print("Calculating closeness ...")

    betweenness = G_ig.edge_betweenness(directed=True, cutoff=None, weights=weight)
    # nx.set_node_attributes(G_nx, closeness1, 'node_closeness')
    # print("Convert to dataframe betweenness ...")
    gdf_edges = ox.utils_graph.graph_to_gdfs(G_nx, nodes=False, edges=True, node_geometry=False, fill_edge_geometry=True)
    df_edges = pd.DataFrame({'osmid': G_ig.es["osmid"], 'edge_betweenness':betweenness})
    gdf_edges = gdf_edges.reset_index(drop=True)
    gdf_res = pd.merge(gdf_edges, df_edges, left_on='osmid', right_on='osmid', how='left')
    # gdf_res.to_postgis("i_street_edge_betw_" + city_name[0:3] , schema=city_name, con=engine)
    gdf_res.to_postgis("street_edge_betw_" + city_name[0:3] , schema=city_name, con=engine)
